Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [29]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic regression

In [22]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(weights)
  loss +=  beta*regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [23]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.332329
Minibatch accuracy: 15.6%
Validation accuracy: 15.7%
Minibatch loss at step 500: 0.741115
Minibatch accuracy: 86.7%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 0.729404
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 0.584653
Minibatch accuracy: 83.6%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.675657
Minibatch accuracy: 86.7%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 0.702339
Minibatch accuracy: 80.5%
Validation accuracy: 80.6%
Minibatch loss at step 3000: 0.796859
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Test accuracy: 88.4%


### Neural network

In [25]:
def prediction(tf_dataset, W1, b1, W2, b2):
    Y1 = tf.matmul(tf_dataset, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    return tf.nn.softmax(Y2)

batch_size = 128
num_nodes = 1024
beta = 1e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
  b1 = tf.Variable(tf.zeros([num_nodes]))
    
  W2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  Y1 = tf.matmul(tf_train_dataset, W1) + b1
  X2 = tf.nn.relu(Y1)
  logits = tf.matmul(X2, W2) + b2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
  loss +=  beta*regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction(tf_valid_dataset, W1, b1, W2, b2)
  test_prediction = prediction(tf_test_dataset, W1, b1, W2, b2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 393.349396
Minibatch accuracy: 11.7%
Validation accuracy: 30.5%
Minibatch loss at step 500: 51.456112
Minibatch accuracy: 82.0%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 36.810547
Minibatch accuracy: 79.7%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 29.921005
Minibatch accuracy: 82.0%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 26.010359
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 27.791925
Minibatch accuracy: 81.2%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 23.349836
Minibatch accuracy: 84.4%
Validation accuracy: 82.9%
Test accuracy: 89.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [26]:
batch_size = 16
num_nodes = 1024

num_train = 100
mini_train_dataset = train_dataset[:num_train]
mini_train_labels = train_labels[:num_train]

num_steps = 1000

def prediction(tf_dataset, W1, b1, W2, b2):
    Y1 = tf.matmul(tf_dataset, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    return tf.nn.softmax(Y2)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
  b1 = tf.Variable(tf.zeros([num_nodes]))
    
  W2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  Y1 = tf.matmul(tf_train_dataset, W1) + b1
  X2 = tf.nn.relu(Y1)
  logits = tf.matmul(X2, W2) + b2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(W1)+ tf.nn.l2_loss(W2)
  loss +=  beta*regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction(tf_valid_dataset, W1, b1, W2, b2)
  test_prediction = prediction(tf_test_dataset, W1, b1, W2, b2)


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (mini_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = mini_train_dataset[offset:(offset + batch_size), :]
    batch_labels = mini_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 283.608490
Minibatch accuracy: 12.5%
Validation accuracy: 20.1%
Minibatch loss at step 100: 32.222389
Minibatch accuracy: 100.0%
Validation accuracy: 56.3%
Minibatch loss at step 200: 31.901567
Minibatch accuracy: 100.0%
Validation accuracy: 56.3%
Minibatch loss at step 300: 31.584476
Minibatch accuracy: 100.0%
Validation accuracy: 56.3%
Minibatch loss at step 400: 31.270300
Minibatch accuracy: 100.0%
Validation accuracy: 56.3%
Minibatch loss at step 500: 30.959166
Minibatch accuracy: 100.0%
Validation accuracy: 56.3%
Minibatch loss at step 600: 30.650906
Minibatch accuracy: 100.0%
Validation accuracy: 56.3%
Minibatch loss at step 700: 30.345928
Minibatch accuracy: 100.0%
Validation accuracy: 56.3%
Minibatch loss at step 800: 30.044022
Minibatch accuracy: 100.0%
Validation accuracy: 56.3%
Minibatch loss at step 900: 29.745174
Minibatch accuracy: 100.0%
Validation accuracy: 56.3%
Test accuracy: 61.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [30]:
def prediction(tf_dataset, W1, b1, W2, b2):
    Y1 = tf.matmul(tf_dataset, W1) + b1
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    return tf.nn.softmax(Y2)

batch_size = 128
num_nodes = 1024
beta = 1e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes], stddev=0.1))
  b1 = tf.Variable(tf.zeros([num_nodes]))
    
  W2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels], stddev=0.1))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  keep_prob = tf.placeholder(tf.float32)
    
  # Training computation.
  Y1 = tf.matmul(tf_train_dataset, W1) + b1
  X2 = tf.nn.relu(Y1)
  X2_drop = tf.nn.dropout(X2, keep_prob)

  logits = tf.matmul(X2_drop, W2) + b2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
  loss += beta*regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction(tf_valid_dataset, W1, b1, W2, b2)
  test_prediction = prediction(tf_test_dataset, W1, b1, W2, b2)

num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.226927
Minibatch accuracy: 14.8%
Validation accuracy: 26.1%
Minibatch loss at step 500: 0.832427
Minibatch accuracy: 85.9%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.939864
Minibatch accuracy: 80.5%
Validation accuracy: 85.2%
Minibatch loss at step 1500: 0.688759
Minibatch accuracy: 88.3%
Validation accuracy: 86.2%
Minibatch loss at step 2000: 0.751130
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 2500: 0.685890
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 3000: 0.718394
Minibatch accuracy: 84.4%
Validation accuracy: 86.7%
Minibatch loss at step 3500: 0.710362
Minibatch accuracy: 84.4%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.680502
Minibatch accuracy: 89.1%
Validation accuracy: 87.6%
Minibatch loss at step 4500: 0.643781
Minibatch accuracy: 88.3%
Validation accuracy: 88.0%
Minibatch loss at step 5000: 0.634077
Minibatch accuracy: 89.8%
Validation accurac

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### 2 hidden layers

In [31]:
def prediction(tf_dataset, W1, b1, W2, b2, W3, b3):
    Y1 = tf.matmul(tf_dataset, W1) + b1
    
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    
    X3 = tf.nn.relu(Y2)
    Y3 = tf.matmul(X3, W3) + b3
    
    return tf.nn.softmax(Y3)

batch_size = 128
num_nodes1 = 1024
num_nodes2 = 512

beta = 1e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes1],stddev=0.1))
  b1 = tf.Variable(tf.zeros([num_nodes1]))
  
  W2 = tf.Variable(tf.truncated_normal([num_nodes1, num_nodes2],stddev=0.1))
  b2 = tf.Variable(tf.zeros([num_nodes2]))

  W3 = tf.Variable(tf.truncated_normal([num_nodes2, num_labels],stddev=0.1))
  b3 = tf.Variable(tf.zeros([num_labels]))
    
  keep_prob = tf.placeholder(tf.float32)
    
  # Training computation.
  Y1 = tf.matmul(tf_train_dataset, W1) + b1
  
  X2 = tf.nn.relu(Y1)
  X2_drop = tf.nn.dropout(X2, keep_prob)
  Y2 = tf.matmul(X2_drop, W2) + b2

  X3 = tf.nn.relu(Y2)
  X3_drop = tf.nn.dropout(X3, keep_prob)

  logits = tf.matmul(X3_drop, W3) + b3
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3)
  loss += beta*regularizers
  
  # Optimizer.
  # optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction(tf_valid_dataset, W1, b1, W2, b2, W3, b3)
  test_prediction = prediction(tf_test_dataset, W1, b1, W2, b2, W3, b3)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.844415
Minibatch accuracy: 8.6%
Validation accuracy: 23.4%
Minibatch loss at step 500: 0.895038
Minibatch accuracy: 86.7%
Validation accuracy: 85.1%
Minibatch loss at step 1000: 0.999021
Minibatch accuracy: 83.6%
Validation accuracy: 86.1%
Minibatch loss at step 1500: 0.751644
Minibatch accuracy: 91.4%
Validation accuracy: 86.9%
Minibatch loss at step 2000: 0.734268
Minibatch accuracy: 93.0%
Validation accuracy: 87.9%
Minibatch loss at step 2500: 0.745342
Minibatch accuracy: 89.8%
Validation accuracy: 88.4%
Minibatch loss at step 3000: 0.842993
Minibatch accuracy: 84.4%
Validation accuracy: 88.2%
Test accuracy: 94.3%


### 3 hidden layers

In [33]:
def prediction(tf_dataset, W1, b1, W2, b2, W3, b3, W4, b4):
    Y1 = tf.matmul(tf_dataset, W1) + b1
    
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    
    X3 = tf.nn.relu(Y2)
    Y3 = tf.matmul(X3, W3) + b3
    
    X4 = tf.nn.relu(Y3)
    Y4 = tf.matmul(X4, W4) + b4
    
    return tf.nn.softmax(Y4)

batch_size = 128
num_nodes1 = 1024
num_nodes2 = 512
num_nodes3 = 64

beta = 5e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes1],stddev=0.1))
  b1 = tf.Variable(tf.zeros([num_nodes1]))
  
  W2 = tf.Variable(tf.truncated_normal([num_nodes1, num_nodes2],stddev=0.1))
  b2 = tf.Variable(tf.zeros([num_nodes2]))

  W3 = tf.Variable(tf.truncated_normal([num_nodes2, num_nodes3],stddev=0.1))
  b3 = tf.Variable(tf.zeros([num_nodes3]))

  W4 = tf.Variable(tf.truncated_normal([num_nodes3, num_labels],stddev=0.1))
  b4 = tf.Variable(tf.zeros([num_labels]))
    
  keep_prob = tf.placeholder(tf.float32)
    
  # Training computation.
  Y1 = tf.matmul(tf_train_dataset, W1) + b1
  
  X2 = tf.nn.relu(Y1)
  X2_drop = tf.nn.dropout(X2, keep_prob)
  Y2 = tf.matmul(X2_drop, W2) + b2
  
  X3 = tf.nn.relu(Y2)
  X3_drop = tf.nn.dropout(X3, keep_prob)
  Y3 = tf.matmul(X3_drop, W3) + b3

  X4 = tf.nn.relu(Y3)
  X4_drop = tf.nn.dropout(X4, keep_prob)

  logits = tf.matmul(X4_drop, W4) + b4
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularizers = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3) + tf.nn.l2_loss(W4)

  loss +=  beta*regularizers
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = prediction(tf_valid_dataset, W1, b1, W2, b2, W3, b3, W4, b4)
  test_prediction = prediction(tf_test_dataset, W1, b1, W2, b2, W3, b3, W4, b4)

num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 9.664207
Minibatch accuracy: 10.2%
Validation accuracy: 12.2%
Minibatch loss at step 5000: 0.945761
Minibatch accuracy: 86.7%
Validation accuracy: 87.2%
Minibatch loss at step 10000: 0.702300
Minibatch accuracy: 85.9%
Validation accuracy: 88.5%
Minibatch loss at step 15000: 0.532235
Minibatch accuracy: 88.3%
Validation accuracy: 89.4%
Minibatch loss at step 20000: 0.614385
Minibatch accuracy: 85.9%
Validation accuracy: 89.9%
Minibatch loss at step 25000: 0.515885
Minibatch accuracy: 88.3%
Validation accuracy: 90.3%
Minibatch loss at step 30000: 0.387454
Minibatch accuracy: 92.2%
Validation accuracy: 90.8%
Test accuracy: 96.0%
